# Einfacher Bildklassifikator

Dieses Notebook zeigt, wie man Bilder mit einem vortrainierten neuronalen Netzwerk klassifiziert.

**Was Sie lernen werden:**
- Wie man ein vortrainiertes Modell lädt und verwendet
- Bildvorverarbeitung
- Vorhersagen für Bilder treffen
- Verständnis von Vertrauenswerten

**Anwendungsfall:** Objekte in Bildern identifizieren (wie "Katze", "Hund", "Auto" usw.)

---


## Schritt 1: Erforderliche Bibliotheken importieren

Lassen Sie uns die benötigten Tools importieren. Keine Sorge, wenn Sie noch nicht alles davon verstehen!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Schritt 2: Vortrainiertes Modell laden

Wir verwenden **MobileNetV2**, ein neuronales Netzwerk, das bereits mit Millionen von Bildern trainiert wurde.

Das nennt man **Transfer Learning** – die Nutzung eines Modells, das von jemand anderem trainiert wurde!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Schritt 3: Hilfsfunktionen

Lassen Sie uns Funktionen erstellen, um Bilder für unser Modell zu laden und vorzubereiten.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Schritt 4: Testen mit Beispielbildern

Lass uns einige Bilder aus dem Internet klassifizieren!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Klassifizieren Sie jedes Bild


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Schritt 5: Probieren Sie Ihre eigenen Bilder aus!

Ersetzen Sie die untenstehende URL durch eine beliebige Bild-URL, die Sie klassifizieren möchten.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Was ist gerade passiert?

1. **Wir haben ein vortrainiertes Modell geladen** - MobileNetV2 wurde mit Millionen von Bildern trainiert.  
2. **Wir haben Bilder vorverarbeitet** - Sie wurden für das Modell passend skaliert und formatiert.  
3. **Das Modell hat Vorhersagen getroffen** - Es hat Wahrscheinlichkeiten für 1000 Objektklassen ausgegeben.  
4. **Wir haben die Ergebnisse dekodiert** - Zahlen wurden in menschenlesbare Labels umgewandelt.  

### Verständnis der Vertrauenswerte

- **90-100%**: Sehr sicher (fast garantiert korrekt)  
- **70-90%**: Sicher (wahrscheinlich korrekt)  
- **50-70%**: Etwas sicher (könnte korrekt sein)  
- **Unter 50%**: Nicht sehr sicher (unsicher)  

### Warum könnten Vorhersagen falsch sein?

- **Ungewöhnlicher Winkel oder Beleuchtung** - Das Modell wurde mit typischen Fotos trainiert.  
- **Mehrere Objekte** - Das Modell erwartet ein Hauptobjekt.  
- **Seltene Objekte** - Das Modell kennt nur 1000 Kategorien.  
- **Schlechte Bildqualität** - Verschwommene oder verpixelte Bilder sind schwieriger zu erkennen.  

---


## 🚀 Nächste Schritte

1. **Probieren Sie verschiedene Bilder aus:**
   - Finden Sie Bilder auf [Unsplash](https://unsplash.com)
   - Rechtsklick → „Bildadresse kopieren“, um die URL zu erhalten

2. **Experimentieren:**
   - Was passiert mit abstrakter Kunst?
   - Kann es Objekte aus verschiedenen Blickwinkeln erkennen?
   - Wie geht es mit mehreren Objekten um?

3. **Mehr erfahren:**
   - Erkunden Sie [Computer Vision Lektionen](../lessons/4-ComputerVision/README.md)
   - Lernen Sie, Ihren eigenen Bildklassifikator zu trainieren
   - Verstehen Sie, wie CNNs (Convolutional Neural Networks) funktionieren

---

## 🎉 Herzlichen Glückwunsch!

Sie haben gerade einen Bildklassifikator mit einem hochmodernen neuronalen Netzwerk erstellt!

Diese Technik wird verwendet für:
- Google Fotos (Organisieren Ihrer Bilder)
- Selbstfahrende Autos (Objekterkennung)
- Medizinische Diagnosen (Analyse von Röntgenbildern)
- Qualitätskontrolle (Erkennung von Defekten)

Bleiben Sie neugierig und lernen Sie weiter! 🚀



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, weisen wir darauf hin, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die aus der Nutzung dieser Übersetzung entstehen.
